단어 시퀀스에 확률을 할당하는 모델

N-gram 언어 모델 
- 일부 단어만 고려하는 접근 방법
- 일부 단어 몇 개 보느냐 = n
- 전체 문장을 고려한 언어 모델보다는 정확도가 떨어짐
- 한계점
    - 희소 문제
    - n 선택하는 것은 trade-off문제
    

# 한국어에서의 언어모델
1. 한국어는 어순이 중요하지 않다
    - 말 통함
2. 한국어는 교착어
    - 조사가 있다. 
    - 토큰화를 통해 접사나 조사 등 분리하는 것이 중요한 작업
3. 띄어쓰기가 제대로 지켜지지 않음

# 펄플렉서티
- 모델 뭐가 더 좋은지 외부 평가 
- 모델 내에서 자신의 성능을 수치화하여 결과를 내놓는 내부평가

1. 언어 모델 평가 방법 : ppl
    - 낮을 수록 좋음
2. 분기 계수
    - 선택할 수 있는 가능한 경우의 수
    

# 카운트 기반의 단어 표현
## 1. 단어의 표현 방법
## 2. Bag of Words(BoW)

In [1]:
from konlpy.tag import Okt
import re  
okt=Okt()  

token=re.sub("(\.)","","정부가 발표하는 물가상승률과 소비자가 느끼는 물가상승률은 다르다.")  
# 정규 표현식을 통해 온점을 제거하는 정제 작업입니다.  
token=okt.morphs(token)  
# OKT 형태소 분석기를 통해 토큰화 작업을 수행한 뒤에, token에다가 넣습니다.  

word2index={}  
bow=[]  
for voca in token:  
         if voca not in word2index.keys():  
             word2index[voca]=len(word2index)  
# token을 읽으면서, word2index에 없는 (not in) 단어는 새로 추가하고, 이미 있는 단어는 넘깁니다.   
             bow.insert(len(word2index)-1,1)
# BoW 전체에 전부 기본값 1을 넣어줍니다. 단어의 개수는 최소 1개 이상이기 때문입니다.  
         else:
            index=word2index.get(voca)
# 재등장하는 단어의 인덱스를 받아옵니다.
            bow[index]=bow[index]+1
# 재등장한 단어는 해당하는 인덱스의 위치에 1을 더해줍니다. (단어의 개수를 세는 것입니다.)  
print(word2index)  

{'정부': 0, '가': 1, '발표': 2, '하는': 3, '물가상승률': 4, '과': 5, '소비자': 6, '느끼는': 7, '은': 8, '다르다': 9}


In [2]:
bow

[1, 2, 1, 1, 2, 1, 1, 1, 1, 1]

In [4]:
from sklearn.feature_extraction.text import CountVectorizer
corpus = ['you know I want your love. because I love you.']
vector = CountVectorizer()
print(vector.fit_transform(corpus).toarray()) # 코퍼스로부터 각 단어의 빈도 수를 기록한다.
print(vector.vocabulary_) # 각 단어의 인덱스가 어떻게 부여되었는지를 보여준다.

[[1 1 2 1 2 1]]
{'you': 4, 'know': 1, 'want': 3, 'your': 5, 'love': 2, 'because': 0}


In [5]:
# 불용어를 제거한 BoW 만들기

#사용자가 지정한 불용어
from sklearn.feature_extraction.text import CountVectorizer

text=["Family is not an important thing. It's everything."]
vect = CountVectorizer(stop_words=["the", "a", "an", "is", "not"])
print(vect.fit_transform(text).toarray()) 
print(vect.vocabulary_)

[[1 1 1 1 1]]
{'family': 1, 'important': 2, 'thing': 4, 'it': 3, 'everything': 0}


In [6]:
#CountVectorizer에서 제공하는 자체 불용어 사용
from sklearn.feature_extraction.text import CountVectorizer

text=["Family is not an important thing. It's everything."]
vect = CountVectorizer(stop_words="english")
print(vect.fit_transform(text).toarray())
print(vect.vocabulary_)

[[1 1 1]]
{'family': 0, 'important': 1, 'thing': 2}


In [7]:
# NLTK에서 지원하는 불용어 사용
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords

text=["Family is not an important thing. It's everything."]
sw = stopwords.words("english")
vect = CountVectorizer(stop_words =sw)
print(vect.fit_transform(text).toarray()) 
print(vect.vocabulary_)

[[1 1 1 1]]
{'family': 1, 'important': 2, 'thing': 3, 'everything': 0}


TF-IDF : DTM 내에 있는 각 단어에 대한 중요도 계산
- DTM만듦
- TF-IDF 가중치 부여
- 특정 문서에만 자주 등장하는 단어는 중요도가 높다고 판단. 
- 자주 등장하는 단어는 중요도 낮음


- td : 특정 문서 d에서 특정 단어 t의 등장 횟수
- df : 특정 단어 t가 등장한 문서의 수
- idf : df에 반비례하는 수

In [8]:
import pandas as pd # 데이터프레임 사용을 위해
from math import log # IDF 계산을 위해

In [9]:
docs = [
  '먹고 싶은 사과',
  '먹고 싶은 바나나',
  '길고 노란 바나나 바나나',
  '저는 과일이 좋아요'
] 
vocab = list(set(w for doc in docs for w in doc.split()))
vocab.sort()

In [10]:
N = len(docs) # 총 문서의 수

def tf(t, d):
    return d.count(t)

def idf(t):
    df = 0
    for doc in docs:
        df += t in doc
    return log(N/(df + 1))

def tfidf(t, d):
    return tf(t,d)* idf(t)

In [11]:
result = []
for i in range(N): # 각 문서에 대해서 아래 명령을 수행
    result.append([])
    d = docs[i]
    for j in range(len(vocab)):
        t = vocab[j]        
        result[-1].append(tf(t, d))

tf_ = pd.DataFrame(result, columns = vocab)
tf_

,과일이,길고,노란,먹고,바나나,사과,싶은,저는,좋아요
0,0,0,0,1,0,1,1,0,0
1,0,0,0,1,1,0,1,0,0
2,0,1,1,0,2,0,0,0,0
3,1,0,0,0,0,0,0,1,1


In [12]:
result = []
for j in range(len(vocab)):
    t = vocab[j]
    result.append(idf(t))

idf_ = pd.DataFrame(result, index = vocab, columns = ["IDF"])
idf_

,IDF
과일이,0.693147
길고,0.693147
노란,0.693147
먹고,0.287682
바나나,0.287682
사과,0.693147
싶은,0.287682
저는,0.693147
좋아요,0.693147


In [15]:
#실제로는 +1해줌
result = []
for i in range(N):
    result.append([])
    d = docs[i]
    for j in range(len(vocab)):
        t = vocab[j]

        result[-1].append(tfidf(t,d))

tfidf_ = pd.DataFrame(result, columns = vocab)
tfidf_

,과일이,길고,노란,먹고,바나나,사과,싶은,저는,좋아요
0,0.000000,0.000000,0.000000,0.287682,0.000000,0.693147,0.287682,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.287682,0.287682,0.000000,0.287682,0.000000,0.000000
2,0.000000,0.693147,0.693147,0.000000,0.575364,0.000000,0.000000,0.000000,0.000000
3,0.693147,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.693147,0.693147


In [17]:
# 사이킷런으로 실습
from sklearn.feature_extraction.text import CountVectorizer
corpus = [
    'you know I want your love',
    'I like you',
    'what should I do ',    
]
vector = CountVectorizer()
print(vector.fit_transform(corpus).toarray()) # 코퍼스로부터 각 단어의 빈도 수를 기록한다.
print(vector.vocabulary_) # 각 단어의 인덱스가 어떻게 부여되었는지를 보여준다.

[[0 1 0 1 0 1 0 1 1]
 [0 0 1 0 0 0 0 1 0]
 [1 0 0 0 1 0 1 0 0]]
{'you': 7, 'know': 1, 'want': 5, 'your': 8, 'love': 3, 'like': 2, 'what': 6, 'should': 4, 'do': 0}


In [19]:
# 자동 계산해주는 TfidfVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
corpus = [
    'you know I want your love',
    'I like you',
    'what should I do ',    
]
tfidfv = TfidfVectorizer().fit(corpus)
print(tfidfv.transform(corpus).toarray())
print(tfidfv.vocabulary_)

[[0.         0.46735098 0.         0.46735098 0.         0.46735098
  0.         0.35543247 0.46735098]
 [0.         0.         0.79596054 0.         0.         0.
  0.         0.60534851 0.        ]
 [0.57735027 0.         0.         0.         0.57735027 0.
  0.57735027 0.         0.        ]]
{'you': 7, 'know': 1, 'want': 5, 'your': 8, 'love': 3, 'like': 2, 'what': 6, 'should': 4, 'do': 0}


# 5. 벡터의 유사도
## 5-1. 코사인 유사도

In [20]:
from numpy import dot
from numpy.linalg import norm
import numpy as np
def cos_sim(A, B):
       return dot(A, B)/(norm(A)*norm(B))

In [22]:
doc1=np.array([0,1,1,1])
doc2=np.array([1,0,1,1])
doc3=np.array([2,0,2,2])
print(cos_sim(doc1, doc2)) #문서1과 문서2의 코사인 유사도
print(cos_sim(doc1, doc3)) #문서1과 문서3의 코사인 유사도
print(cos_sim(doc2, doc3)) #문서2과 문서3의 코사인 유사도

0.6666666666666667
0.6666666666666667
1.0000000000000002


In [23]:
# 영화 추천 시스템
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [24]:
data = pd.read_csv('movies_metadata.csv', low_memory=False)
data.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


In [25]:
#데이터 량 재저장
data = data.head(20000)

In [26]:
data['overview'].isnull().sum()

135

In [27]:
# overview에서 Null 값을 가진 경우에는 Null 값을 제거
data['overview'] = data['overview'].fillna('')

In [28]:
tfidf = TfidfVectorizer(stop_words='english')
# overview에 대해서 tf-idf 수행
tfidf_matrix = tfidf.fit_transform(data['overview'])
print(tfidf_matrix.shape)

(20000, 47487)


In [33]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [34]:
#코사인 유사도
indices = pd.Series(data.index, index=data['title']).drop_duplicates()
print(indices.head())

title
Toy Story                      0
Jumanji                        1
Grumpier Old Men               2
Waiting to Exhale              3
Father of the Bride Part II    4
dtype: int64


In [30]:
indices

title
Toy Story                                                                       0
Jumanji                                                                         1
Grumpier Old Men                                                                2
Waiting to Exhale                                                               3
Father of the Bride Part II                                                     4
                                                                            ...  
Rebellion                                                                   19995
Versailles                                                                  19996
Two in the Wave                                                             19997
Lotte Reiniger: Homage to the Inventor of the Silhouette Film               19998
RKO Production 601: The Making of 'Kong, the Eighth Wonder of the World'    19999
Length: 20000, dtype: int64

In [35]:
idx = indices['Father of the Bride Part II']
print(idx)

4


In [36]:
# overview가 유사한 10개의 영화 찾아내기
def get_recommendations(title, cosine_sim=cosine_sim):
    # 선택한 영화의 타이틀로부터 해당되는 인덱스를 받아옵니다. 이제 선택한 영화를 가지고 연산할 수 있습니다.
    idx = indices[title]

    # 모든 영화에 대해서 해당 영화와의 유사도를 구합니다.
    sim_scores = list(enumerate(cosine_sim[idx]))

    # 유사도에 따라 영화들을 정렬합니다.
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # 가장 유사한 10개의 영화를 받아옵니다.
    sim_scores = sim_scores[1:11]

    # 가장 유사한 10개의 영화의 인덱스를 받아옵니다.
    movie_indices = [i[0] for i in sim_scores]

    # 가장 유사한 10개의 영화의 제목을 리턴합니다.
    return data['title'].iloc[movie_indices]

In [37]:
get_recommendations('The Dark Knight Rises')

12481                            The Dark Knight
150                               Batman Forever
1328                              Batman Returns
15511                 Batman: Under the Red Hood
585                                       Batman
9230          Batman Beyond: Return of the Joker
18035                           Batman: Year One
19792    Batman: The Dark Knight Returns, Part 1
3095                Batman: Mask of the Phantasm
10122                              Batman Begins
Name: title, dtype: object

## 5-2 여러가지 유사도 기법

In [38]:
# 유클리드 거리
import numpy as np
def dist(x,y):   
    return np.sqrt(np.sum((x-y)**2))

doc1 = np.array((2,3,0,1))
doc2 = np.array((1,2,3,1))
doc3 = np.array((2,1,2,2))
docQ = np.array((1,1,0,1))

print(dist(doc1,docQ))
print(dist(doc2,docQ))
print(dist(doc3,docQ))

2.23606797749979
3.1622776601683795
2.449489742783178


In [39]:
# 자카드 유사도
# 두 문서 모두에서 등장한 단어는 apple과 banana 2개.
doc1 = "apple banana everyone like likey watch card holder"
doc2 = "apple banana coupon passport love you"

# 토큰화를 수행합니다.
tokenized_doc1 = doc1.split()
tokenized_doc2 = doc2.split()

# 토큰화 결과 출력
print(tokenized_doc1)
print(tokenized_doc2)

['apple', 'banana', 'everyone', 'like', 'likey', 'watch', 'card', 'holder']
['apple', 'banana', 'coupon', 'passport', 'love', 'you']


In [40]:
#합집합
union = set(tokenized_doc1).union(set(tokenized_doc2))
print(union)

{'passport', 'card', 'love', 'watch', 'coupon', 'likey', 'you', 'apple', 'banana', 'holder', 'everyone', 'like'}


In [41]:
# 교집합
intersection = set(tokenized_doc1).intersection(set(tokenized_doc2))
print(intersection)

{'apple', 'banana'}


In [42]:
# 자카드 유사도
print(len(intersection)/len(union)) # 2를 12로 나눔.

0.16666666666666666


# 6. 토픽 모델링
텍스트 본문의 숨겨진 의미 구조를 발견
## 6-1 잠재 의미 분석(LSA)
- LDA : LSA의 단점 개선
### (1) 특이값 분해 (SVD)

In [43]:
import numpy as np
A=np.array([[0,0,0,1,0,1,1,0,0],[0,0,0,1,1,0,1,0,0],[0,1,1,0,2,0,0,0,0],[1,0,0,0,0,0,0,1,1]])
np.shape(A)

(4, 9)

In [44]:
# 변수명 : s, 전치 행렬 : VT
U, s, VT = np.linalg.svd(A, full_matrices = True)

In [45]:
print(U.round(2))
np.shape(U)

[[-0.24  0.75  0.   -0.62]
 [-0.51  0.44 -0.    0.74]
 [-0.83 -0.49 -0.   -0.27]
 [-0.   -0.    1.    0.  ]]


(4, 4)

In [46]:
print(s.round(2))
np.shape(s)

[2.69 2.05 1.73 0.77]


(4,)

In [47]:
S = np.zeros((4, 9)) # 대각 행렬의 크기인 4 x 9의 임의의 행렬 생성
S[:4, :4] = np.diag(s) # 특이값을 대각행렬에 삽입
print(S.round(2))
np.shape(S)

[[2.69 0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   2.05 0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   1.73 0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.77 0.   0.   0.   0.   0.  ]]


(4, 9)

In [48]:
print(VT.round(2))
np.shape(VT)

[[-0.   -0.31 -0.31 -0.28 -0.8  -0.09 -0.28 -0.   -0.  ]
 [ 0.   -0.24 -0.24  0.58 -0.26  0.37  0.58 -0.   -0.  ]
 [ 0.58 -0.    0.    0.   -0.    0.   -0.    0.58  0.58]
 [ 0.   -0.35 -0.35  0.16  0.25 -0.8   0.16 -0.   -0.  ]
 [-0.   -0.78 -0.01 -0.2   0.4   0.4  -0.2   0.    0.  ]
 [-0.29  0.31 -0.78 -0.24  0.23  0.23  0.01  0.14  0.14]
 [-0.29 -0.1   0.26 -0.59 -0.08 -0.08  0.66  0.14  0.14]
 [-0.5  -0.06  0.15  0.24 -0.05 -0.05 -0.19  0.75 -0.25]
 [-0.5  -0.06  0.15  0.24 -0.05 -0.05 -0.19 -0.25  0.75]]


(9, 9)

In [49]:
np.allclose(A, np.dot(np.dot(U,S), VT).round(2))

True

#### 절단된 SVD

In [50]:
S=S[:2,:2]
print(S.round(2))

[[2.69 0.  ]
 [0.   2.05]]


In [53]:
U=U[:,:2]
print(U.round(2))

[[-0.24  0.75]
 [-0.51  0.44]
 [-0.83 -0.49]
 [-0.   -0.  ]]


In [56]:
VT=VT[:2,:]
print(VT.round(2))

[[-0.   -0.31 -0.31 -0.28 -0.8  -0.09 -0.28 -0.   -0.  ]
 [ 0.   -0.24 -0.24  0.58 -0.26  0.37  0.58 -0.   -0.  ]]


In [57]:
# 값 손실돼서 다른 결과 값 나옴
A_prime=np.dot(np.dot(U,S), VT)
print(A)
print(A_prime.round(2))

[[0 0 0 1 0 1 1 0 0]
 [0 0 0 1 1 0 1 0 0]
 [0 1 1 0 2 0 0 0 0]
 [1 0 0 0 0 0 0 1 1]]
[[ 0.   -0.17 -0.17  1.08  0.12  0.62  1.08 -0.   -0.  ]
 [ 0.    0.2   0.2   0.91  0.86  0.45  0.91  0.    0.  ]
 [ 0.    0.93  0.93  0.03  2.05 -0.17  0.03  0.    0.  ]
 [ 0.    0.    0.    0.    0.   -0.    0.    0.    0.  ]]


### 뉴스그룹 데이터 실습 (LSA 사용)
- 문서의 수 --> 원하는 토픽의 수로 압축
- 각 토픽당 가장 중요한 단어 5개 출력

In [58]:
import pandas as pd
from sklearn.datasets import fetch_20newsgroups
dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove=('headers', 'footers', 'quotes'))
documents = dataset.data
len(documents)

11314

In [59]:
documents[1]

"\n\n\n\n\n\n\nYeah, do you expect people to read the FAQ, etc. and actually accept hard\natheism?  No, you need a little leap of faith, Jimmy.  Your logic runs out\nof steam!\n\n\n\n\n\n\n\nJim,\n\nSorry I can't pity you, Jim.  And I'm sorry that you have these feelings of\ndenial about the faith you need to get by.  Oh well, just pretend that it will\nall end happily ever after anyway.  Maybe if you start a new newsgroup,\nalt.atheist.hard, you won't be bummin' so much?\n\n\n\n\n\n\nBye-Bye, Big Jim.  Don't forget your Flintstone's Chewables!  :) \n--\nBake Timmons, III"

In [60]:
#카테고리
print(dataset.target_names)

['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


In [61]:
# 텍스트 전처리
news_df = pd.DataFrame({'document':documents})
# 특수 문자 제거
news_df['clean_doc'] = news_df['document'].str.replace("[^a-zA-Z]", " ")
# 길이가 3이하인 단어는 제거 (길이가 짧은 단어 제거)
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
# 전체 단어에 대한 소문자 변환
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: x.lower())

In [62]:
news_df['clean_doc'][1]

'yeah expect people read actually accept hard atheism need little leap faith jimmy your logic runs steam sorry pity sorry that have these feelings denial about faith need well just pretend that will happily ever after anyway maybe start newsgroup atheist hard bummin much forget your flintstone chewables bake timmons'

In [63]:
# 불용어 제거
from nltk.corpus import stopwords
stop_words = stopwords.words('english') # NLTK로부터 불용어를 받아옵니다.
tokenized_doc = news_df['clean_doc'].apply(lambda x: x.split()) # 토큰화
tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words])


In [64]:
print(tokenized_doc[1])

['yeah', 'expect', 'people', 'read', 'actually', 'accept', 'hard', 'atheism', 'need', 'little', 'leap', 'faith', 'jimmy', 'logic', 'runs', 'steam', 'sorry', 'pity', 'sorry', 'feelings', 'denial', 'faith', 'need', 'well', 'pretend', 'happily', 'ever', 'anyway', 'maybe', 'start', 'newsgroup', 'atheist', 'hard', 'bummin', 'much', 'forget', 'flintstone', 'chewables', 'bake', 'timmons']


In [65]:
# TF-IDF 행렬 만들기 위해 토큰화 취소 
# 역토큰화 (토큰화 작업을 역으로 되돌림)
detokenized_doc = []
for i in range(len(news_df)):
    t = ' '.join(tokenized_doc[i])
    detokenized_doc.append(t)

news_df['clean_doc'] = detokenized_doc

In [66]:
news_df['clean_doc'][1]

'yeah expect people read actually accept hard atheism need little leap faith jimmy logic runs steam sorry pity sorry feelings denial faith need well pretend happily ever anyway maybe start newsgroup atheist hard bummin much forget flintstone chewables bake timmons'

In [ ]:
# 1000개에 대한 TF-IDF 행렬 만듦

In [67]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(stop_words='english', 
max_features= 1000, # 상위 1,000개의 단어를 보존 
max_df = 0.5, 
smooth_idf=True)

X = vectorizer.fit_transform(news_df['clean_doc'])
X.shape # TF-IDF 행렬의 크기 확인

(11314, 1000)

In [69]:
# 토픽 모델링
# TF-IDF --> 다수의 행렬로 분해
from sklearn.decomposition import TruncatedSVD
svd_model = TruncatedSVD(n_components=20, algorithm='randomized', n_iter=100, random_state=122)
svd_model.fit(X)
len(svd_model.components_)

20

In [70]:
# LSA의 VT
np.shape(svd_model.components_)

(20, 1000)

In [71]:
terms = vectorizer.get_feature_names() # 단어 집합. 1,000개의 단어가 저장됨.

def get_topics(components, feature_names, n=5):
    for idx, topic in enumerate(components):
        print("Topic %d:" % (idx+1), [(feature_names[i], topic[i].round(5)) for i in topic.argsort()[:-n - 1:-1]])
get_topics(svd_model.components_,terms)

Topic 1: [('like', 0.21378), ('know', 0.20032), ('people', 0.19317), ('think', 0.17807), ('good', 0.15105)]
Topic 2: [('thanks', 0.3292), ('windows', 0.29094), ('card', 0.18016), ('drive', 0.1739), ('mail', 0.15126)]
Topic 3: [('game', 0.37161), ('team', 0.32553), ('year', 0.28204), ('games', 0.25416), ('season', 0.18464)]
Topic 4: [('drive', 0.52804), ('scsi', 0.20035), ('disk', 0.15514), ('hard', 0.15506), ('card', 0.14041)]
Topic 5: [('windows', 0.4052), ('file', 0.25609), ('window', 0.18057), ('files', 0.1619), ('program', 0.1401)]
Topic 6: [('chip', 0.16138), ('government', 0.16026), ('mail', 0.15625), ('space', 0.1505), ('information', 0.13575)]
Topic 7: [('like', 0.67109), ('bike', 0.14268), ('know', 0.11428), ('chip', 0.11058), ('sounds', 0.10398)]
Topic 8: [('card', 0.45115), ('sale', 0.21607), ('video', 0.21391), ('monitor', 0.14913), ('offer', 0.14872)]
Topic 9: [('know', 0.44979), ('card', 0.35491), ('chip', 0.17206), ('video', 0.15184), ('government', 0.15053)]
Topic 10: [

In [73]:
svd_model.components_

array([[ 0.01470539,  0.05013962,  0.02134918, ...,  0.07873737,
         0.014319  ,  0.01790916],
       [-0.00534936,  0.01658066, -0.01646747, ..., -0.06345616,
        -0.01060612, -0.01904113],
       [ 0.0017196 , -0.00369764, -0.01805701, ...,  0.05886974,
         0.02633566,  0.02242595],
       ...,
       [-0.01123349,  0.00438157,  0.00291653, ...,  0.02019132,
        -0.00092632,  0.00045298],
       [ 0.00178685,  0.01482044,  0.01102556, ..., -0.0905083 ,
        -0.00139842, -0.00541324],
       [ 0.00188385, -0.03627354, -0.00572834, ...,  0.03669636,
        -0.01423495, -0.00350786]])

LSA의 장점
- SVD의 특성상 이미 계산된 LSA에 새로운 데이터 추가하여 계산하려면 처음부터 계산해야함
- 새로운 정보에 대해 업데이트 어려움


## 6-2 잠재 디리클레 할당(LDA)
### (1) 잠재 디리클레 할당
- 문서 집합으로부터 어떤 토픽이 존재하는지를 알아내기 위한 알고리즘

- BoW(빈도수 기반의 표현방법)의 행렬 DTM 
- TF-IDF행렬을 입력으로

- 단어의 순서는 신경 X

### 수행 과정
- 토픽의 개수 K 알려줌
- 모든 단어를 K개 중 하나의 토픽에 할당함(랜덤으로)
    - 각 문서는 토픽을 가짐
    - 랜덤이라서 결과는 다 틀린 상태


### 잠재 디리클레 할당 != 잠재 의미 분석
- LDA : 단어가 특정 토픽에 존재할 확률과 문서에 특정 토픽이 존재할 확률을 결합확률로 추정하여 토픽 추출
- LSA : DTM을 차원 축소하여 축소 차원에서 근접 단어들을 토픽으로 묶음

In [76]:
#실습

In [110]:
tokenized_doc[:5]

0    [well, sure, story, seem, biased, disagree, st...
1    [yeah, expect, people, read, actually, accept,...
2    [although, realize, principle, strongest, poin...
3    [notwithstanding, legitimate, fuss, proposal, ...
4    [well, change, scoring, playoff, pool, unfortu...
Name: clean_doc, dtype: object

In [111]:
# [(정수 인코딩된 값, 단어의 빈도수)]
from gensim import corpora
dictionary = corpora.Dictionary(tokenized_doc)
corpus = [dictionary.doc2bow(text) for text in tokenized_doc]
print(corpus[1]) # 수행된 결과에서 두번째 뉴스 출력. 첫번째 문서의 인덱스는 0

[(52, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 2), (67, 1), (68, 1), (69, 1), (70, 1), (71, 2), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 2), (79, 1), (80, 1), (81, 1), (82, 1), (83, 1), (84, 1), (85, 2), (86, 1), (87, 1), (88, 1), (89, 1)]


In [112]:
print(dictionary[66])

faith


In [113]:
len(dictionary)

64281

In [114]:
# LDA 모델 훈련시키기

# 토픽에 대한 기여도 보여줌
# passes --> 알고리즘 동작 횟수
import gensim
NUM_TOPICS = 20 #20개의 토픽, k=20
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
topics = ldamodel.print_topics(num_words=4)#4개의 단어만
for topic in topics:
    print(topic)

(0, '0.013*"jesus" + 0.010*"would" + 0.010*"people" + 0.007*"bible"')
(1, '0.020*"wire" + 0.020*"ground" + 0.013*"circuit" + 0.012*"power"')
(2, '0.065*"turkish" + 0.047*"turkey" + 0.042*"greek" + 0.031*"turks"')
(3, '0.019*"team" + 0.016*"play" + 0.015*"game" + 0.014*"hockey"')
(4, '0.010*"smokeless" + 0.008*"music" + 0.008*"rockefeller" + 0.007*"wave"')
(5, '0.021*"would" + 0.015*"like" + 0.013*"think" + 0.011*"know"')
(6, '0.026*"israel" + 0.026*"jews" + 0.016*"israeli" + 0.015*"jewish"')
(7, '0.023*"health" + 0.021*"medical" + 0.016*"pain" + 0.013*"disease"')
(8, '0.006*"bike" + 0.004*"water" + 0.004*"cars" + 0.004*"high"')
(9, '0.030*"card" + 0.024*"video" + 0.022*"sale" + 0.019*"shipping"')
(10, '0.012*"system" + 0.011*"drive" + 0.009*"would" + 0.008*"know"')
(11, '0.045*"space" + 0.018*"nasa" + 0.010*"launch" + 0.009*"center"')
(12, '0.019*"said" + 0.014*"went" + 0.012*"armenians" + 0.011*"people"')
(13, '0.014*"contest" + 0.012*"myers" + 0.012*"filename" + 0.012*"henrik"')
(14,

In [117]:
pyLDAvis.__version__

'3.3.1'

In [102]:
!pip install pyLDAvis

In [118]:
#LDA 시각화
import pyLDAvis
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)
pyLDAvis.display(vis)

ModuleNotFoundError: No module named 'pyLDAvis.gensim'

In [119]:
for i, topic_list in enumerate(ldamodel[corpus]):
    if i==5:
        break
    print(i,'번째 문서의 topic 비율은',topic_list)

0 번째 문서의 topic 비율은 [(0, 0.33765987), (1, 0.020140452), (2, 0.016984574), (5, 0.11208299), (6, 0.20938297), (13, 0.015852435), (14, 0.16783582), (15, 0.11020687)]
1 번째 문서의 topic 비율은 [(0, 0.26063284), (3, 0.02841772), (5, 0.46006486), (14, 0.20303327), (16, 0.028096378)]
2 번째 문서의 topic 비율은 [(4, 0.033578526), (5, 0.4950483), (6, 0.21210144), (15, 0.24633427)]
3 번째 문서의 topic 비율은 [(1, 0.02066126), (4, 0.048245482), (5, 0.2519364), (8, 0.064942054), (10, 0.3492401), (15, 0.21117564), (18, 0.043613337)]
4 번째 문서의 topic 비율은 [(3, 0.25458777), (5, 0.6377952), (19, 0.07611891)]


In [120]:
def make_topictable_per_doc(ldamodel, corpus):
    topic_table = pd.DataFrame()

    # 몇 번째 문서인지를 의미하는 문서 번호와 해당 문서의 토픽 비중을 한 줄씩 꺼내온다.
    for i, topic_list in enumerate(ldamodel[corpus]):
        doc = topic_list[0] if ldamodel.per_word_topics else topic_list            
        doc = sorted(doc, key=lambda x: (x[1]), reverse=True)
        # 각 문서에 대해서 비중이 높은 토픽순으로 토픽을 정렬한다.
        # EX) 정렬 전 0번 문서 : (2번 토픽, 48.5%), (8번 토픽, 25%), (10번 토픽, 5%), (12번 토픽, 21.5%), 
        # Ex) 정렬 후 0번 문서 : (2번 토픽, 48.5%), (8번 토픽, 25%), (12번 토픽, 21.5%), (10번 토픽, 5%)
        # 48 > 25 > 21 > 5 순으로 정렬이 된 것.

        # 모든 문서에 대해서 각각 아래를 수행
        for j, (topic_num, prop_topic) in enumerate(doc): #  몇 번 토픽인지와 비중을 나눠서 저장한다.
            if j == 0:  # 정렬을 한 상태이므로 가장 앞에 있는 것이 가장 비중이 높은 토픽
                topic_table = topic_table.append(pd.Series([int(topic_num), round(prop_topic,4), topic_list]), ignore_index=True)
                # 가장 비중이 높은 토픽과, 가장 비중이 높은 토픽의 비중과, 전체 토픽의 비중을 저장한다.
            else:
                break
    return(topic_table)

In [121]:
topictable = make_topictable_per_doc(ldamodel, corpus)
topictable = topictable.reset_index() # 문서 번호을 의미하는 열(column)로 사용하기 위해서 인덱스 열을 하나 더 만든다.
topictable.columns = ['문서 번호', '가장 비중이 높은 토픽', '가장 높은 토픽의 비중', '각 토픽의 비중']
topictable[:10]

,문서 번호,가장 비중이 높은 토픽,가장 높은 토픽의 비중,각 토픽의 비중
0,0,0.0,0.3376,"[(0, 0.3376462), (1, 0.02014041), (2, 0.016984..."
1,1,5.0,0.4601,"[(0, 0.26069406), (3, 0.028417703), (5, 0.4600..."
2,2,5.0,0.4951,"[(4, 0.033577863), (5, 0.495051), (6, 0.212103..."
3,3,10.0,0.3492,"[(1, 0.020661924), (4, 0.048245512), (5, 0.251..."
4,4,5.0,0.6378,"[(3, 0.25463918), (5, 0.6378292), (19, 0.07603..."
5,5,0.0,0.3199,"[(0, 0.31994087), (3, 0.07447001), (5, 0.27610..."
6,6,10.0,0.3928,"[(4, 0.040392984), (5, 0.29850543), (8, 0.0261..."
7,7,5.0,0.4702,"[(0, 0.03223405), (5, 0.47022), (6, 0.12821133..."
8,8,5.0,0.2336,"[(0, 0.14686981), (5, 0.23359892), (7, 0.04014..."
9,9,5.0,0.4388,"[(0, 0.019825585), (1, 0.021381484), (5, 0.438..."


## 실습2

In [122]:
import pandas as pd
import urllib.request
urllib.request.urlretrieve("https://raw.githubusercontent.com/franciscadias/data/master/abcnews-date-text.csv", filename="abcnews-date-text.csv")
data = pd.read_csv('abcnews-date-text.csv', error_bad_lines=False)

In [123]:
print(len(data))

1082168


In [124]:
print(data.head(5))

   publish_date                                      headline_text
0      20030219  aba decides against community broadcasting lic...
1      20030219     act fire witnesses must be aware of defamation
2      20030219     a g calls for infrastructure protection summit
3      20030219           air nz staff in aust strike for pay rise
4      20030219      air nz strike to affect australian travellers


In [126]:
text = data[['headline_text']]
text.head(5)

,headline_text
0,aba decides against community broadcasting lic...
1,act fire witnesses must be aware of defamation
2,a g calls for infrastructure protection summit
3,air nz staff in aust strike for pay rise
4,air nz strike to affect australian travellers


In [134]:
from nltk import word_tokenize
word_tokenize("Hello world")

['Hello', 'world']

In [135]:
# 텍스트 전처리
import nltk
text['headline_text'] = text.apply(lambda row: nltk.word_tokenize(row['headline_text']), axis=1)


ImportError: cannot import name 'AggFuncType' from 'pandas._typing' (D:\ProgramData\Anaconda3\lib\site-packages\pandas\_typing.py)